Multi-agent Collaboration for Financial Analysis with Anthropic Claude and CrewAI


In [1]:
#Install libraries

%pip install crewai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install crewai_tools

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install anthropic litellm langchain_community openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [6]:
from crewai import Agent, Task, Crew, LLM

In [6]:
import litellm
from litellm import openai_chat_completions

**Note**:
If you use Google Colab to run you can mount your folder to work with files

In [8]:
# Mount Google Drive. If you use Google Colab this is convenient way to store files and work

from google.colab import drive
drive.mount('/content/drive')

# Add the path to your Google Drive folder to sys.path
import sys
import os

# Replace 'My Drive/Multi-Agency' with the actual path to your Google Drive folder
# where utils.py is located
sys.path.append('/content/drive/My Drive/Multi-Agency')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# USE THIS FOR CLAUDE
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
serper_api_key = get_serper_api_key()

os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL"] = 'gpt-4o-mini'
os.environ["SERPER_API_KEY"] = serper_api_key

In [8]:
# Configure the Claude LLM
llm =LLM(
    model="gpt-4o-mini",
    temperature = 0.7,
    max_tokens = 1000,
    api_key=openai_api_key
)

## crewAI Tools

In [9]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

scrape_tool = ScrapeWebsiteTool()

search_tool = SerperDevTool(
    name="search",
    func="Perform a web search",
    description="Fetches search results from the web",
    n_results=10  # Set the number of results to fetch
)

c:\Users\Ash\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
c:\Users\Ash\AppData\Local\Programs\Python\Python312\Lib\site-packages\crewai_tools\tools\scrapegraph_scrape_tool\scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator("website_url")
c:\Users\Ash\AppData\Local\Programs\Python\Python312\Lib\site-packages\crewai_tools\tools\selenium_scraping

## Creating Agents

In [10]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    llm=llm,
    tools = [scrape_tool, search_tool]
)

In [11]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    llm=llm,
    tools = [scrape_tool, search_tool]
)

In [12]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    llm=llm,
    tools = [scrape_tool, search_tool]
)

In [13]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    llm=llm,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [14]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [15]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [16]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [17]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [18]:
manager_llm = LLM(
    model="gpt-4o-mini",  # You can use different model for Manager LLM
    temperature = 0.7,
    max_tokens = 500,
    api_key=openai_api_key
)

In [19]:
from crewai import Crew, Process

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent,
            trading_strategy_agent,
            execution_agent,
            risk_management_agent],

    tasks=[data_analysis_task,
           strategy_development_task,
           execution_planning_task,
           risk_assessment_task],

    manager_llm=manager_llm,
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [20]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AMZN',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': '1 year',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different every time you run

In [21]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4331fbb6-944c-4e53-b45f-c9b74d426f24                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 2f9e7771-d968-4e6e-bbc7-ac3e49a73ad5
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 2f9e7771-d968-4e6e-bbc7-ac3e49a73ad5
       Status: Executing Task...
    └── 🤖 Agent: Crew Manager
            Status: In Progress

# Agent: Crew Manager
## Task: Continuously monitor and analyze market data for the selected stock (AMZN). Use statistical modeling and machine learning to identify trends and predict market movements.


🚀 Crew: crew
└── 📋 Task: 2f9e7771-d968-4e6e-bbc7-ac3e49a73ad5
       Status: Executing Task...
    └── 🤖 Agent: Crew Manager
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Crew Manager
## Final Answer: 
**Risk Analysis Report for AMZN Trading Strategies**

**Overview**
This report evaluates the risks associated with the proposed trading strategies and execution plans for Amazon.com Inc. (AMZN), detailing potential risks and suggesting mitigation strategies.

**Proposed Trading Strategies**
1. **Bullish Diagonal Spread**
   - **Strategy**: Buy a longer-term call option (e.g., October 2024, strike $190) and sell a shorter-term call option (e.g., May 2024, strike $215).
   - **Potential Risks**:
     - **Limited Profit Potential**: The maximum profit is capped at the difference between the strike prices minus the net cost of the spread.
     - **Earnings Risk**: If AMZN's stock price does not rise as anticipated, the strategy may result in a loss.
   - **Mitigation Strategies**:
     - Set a stop-loss order to limit potential losses.
     - Monitor earnings announcements closely to avoid unfavorable price movements.

2. **Iron Condor**
   - **Str

🚀 Crew: crew
├── 📋 Task: 2f9e7771-d968-4e6e-bbc7-ac3e49a73ad5
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
│       └── 🔧 Used search (1)
├── 📋 Task: 1afe9397-b88d-44ef-a6ea-2f0f6d6f31d9
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Crew Manager
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (1)
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
│       ├── 🔧 Used search (2)
│       └── 🔧 Used Read website content (3)
├── 📋 Task: bba3bc94-f8b6-4e2a-87e3-81dac467bfa2
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
│       ├── 🔧 Used search (3)
│       └── 🔧 Used Read website content (4)
└── 📋 Task: 75d40c4c-bcee-4f25-8a50-0e0c5d02df76
       Status: Executing Task...
    ├── 🤖 Agent: Crew Manager
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (2)
    └── 🤖 Agent: Crew Manager
            Status: ✅ Completed
        ├── 🔧 Used search (5)
        └── 🔧 Used Read website content (7)

🚀 Crew: crew
├── 📋 Task: 2f9e7771-d968-4e6e-bbc7-ac3e49a73ad5
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
│       └── 🔧 Used search (1)
├── 📋 Task: 1afe9397-b88d-44ef-a6ea-2f0f6d6f31d9
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Crew Manager
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (1)
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
│       ├── 🔧 Used search (2)
│       └── 🔧 Used Read website content (3)
├── 📋 Task: bba3bc94-f8b6-4e2a-87e3-81dac467bfa2
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
│       ├── 🔧 Used search (3)
│       └── 🔧 Used Read website content (4)
└── 📋 Task: 75d40c4c-bcee-4f25-8a50-0e0c5d02df76
       Assigned to: Crew Manager
       Status: ✅ Completed
    ├── 🤖 Agent: Crew Manager
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (2)
    └── 🤖 Agent: Crew Manager
            Status: ✅ Completed
        ├── 🔧 Used search (5)
        └── 🔧 Used Read website content (7)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 75d40c4c-bcee-4f25-8a50-0e0c5d02df76                                                                     │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 4331fbb6-944c-4e53-b45f-c9b74d426f24                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Display the final result as Markdown.

In [68]:
from IPython.display import Markdown
Markdown(result.raw)

AMAZON (AMZN) TRADING STRATEGIES: COMPREHENSIVE RISK ANALYSIS REPORT

I. STRATEGIC RISK FRAMEWORK

1. Market Volatility Risks
   a) Macroeconomic Sensitivity Factors
      - High volatility in technology sector
      - Potential economic downturn impacts
      - Interest rate fluctuations affecting investment landscape
   
   b) Portfolio Allocation Risk Profiles
      - Conservative Strategy (5-7%): Lower risk exposure
      - Moderate Strategy (8-12%): Balanced risk-reward
      - Aggressive Strategy (12-15%): Highest potential volatility

2. Technological Innovation Risks
   a) AI Development Challenges
      - Uncertain timeline for AI breakthrough monetization
      - Potential technological obsolescence
      - High investment requirements in emerging technologies
   
   b) Competitive Landscape Risks
      - Intense competition in cloud computing (AWS)
      - Emerging AI technology players
      - Potential market share erosion

3. Regulatory and Compliance Risks
   a) Antitrust Concerns
      - Increasing regulatory scrutiny in US and Europe
      - Potential legal challenges to business model
      - Risk of forced structural changes
   
   b) Global Trade and Tariff Risks
      - Impact of international trade policies
      - Potential margin compression
      - Supply chain disruption vulnerabilities

4. Financial Performance Risks
   a) Revenue Growth Uncertainty
      - Potential slowdown in e-commerce expansion
      - AWS growth rate deceleration
      - Advertising revenue volatility
   
   b) Profitability Challenges
      - High operational expenses
      - Infrastructure investment requirements
      - Potential margin pressure from competitive markets

II. RISK MITIGATION STRATEGIES

1. Dynamic Portfolio Management
   - Maintain 10-15% cash reserve
   - Quarterly strategy reassessment
   - Adaptive risk framework (±15-20% volatility tolerance)

2. Entry and Exit Strategy Safeguards
   a) Conservative Entry Mechanisms
      - Dollar-cost averaging
      - Phased entry points
      - Strategic market opportunity timing
   
   b) Advanced Risk Management Techniques
      - Protective put options
      - Covered call strategies
      - Options spread strategies
      - Momentum-based dynamic exits

3. Performance Monitoring Triggers
   Exit Strategy Considerations:
   - Significant leadership changes
   - Substantial market share loss
   - Adverse regulatory interventions
   - Failure to meet key performance indicators

III. MONITORING AND REASSESSMENT CADENCE
   - Monthly: Portfolio rebalancing
   - Quarterly: Deep-dive performance analysis
   - Annually: Comprehensive strategy review

IV. KEY RISK INDICATORS TO TRACK
   1. AWS Growth Metrics
   2. E-commerce Performance
   3. AI Innovation Progress
   4. Advertising Revenue Expansion
   5. Technological Breakthrough Announcements
   6. Market Expansion Metrics

V. RECOMMENDED APPROACH
Implement a dynamic, multi-tiered strategy with:
- Continuous monitoring
- Adaptive risk management
- Segmented entry points
- Maintained flexibility
- Focus on core growth drivers

CONCLUSION:
While Amazon presents significant growth potential, the proposed strategies require rigorous, dynamic management to navigate the complex technological and market landscape. Success depends on continuous adaptation, strategic flexibility, and proactive risk mitigation.